In [22]:
import tensorflow as tf
import prettytensor as pt
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import math
import os
import time
from datetime import timedelta

import helper_plot
from helper_plot import plot_images

%matplotlib inline

In [27]:
from tensorflow.examples.tutorials.mnist import input_data

#net configuration
#conv layer1
filter_size1 = 5
num_filters1 = 16
#conv layer2
filter_size2 = 5
num_filters2 = 36
#dense layer1
dense_size = 128

#images configuration
img_size = 28
img_size_flat = img_size*img_size
img_shape = (img_size,img_size)
num_channels = 1
num_classes = 10


data = input_data.read_data_sets('data/MNIST/', one_hot=True)
data.test.cls = np.argmax(data.test.labels, axis=1)

print("Size of:")
print("- Training set:\t\t{}".format(len(data.train.labels)))
print("- Test set:\t\t{}".format(len(data.test.labels)))
print("- Validation set:\t{}".format(len(data.validation.labels)))

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
- Training set:		55000
- Test set:		10000
- Validation set:	5000


In [28]:
x = tf.placeholder(tf.float32, shape=[None,img_size_flat], name='x')
x_images = tf.reshape(x, [-1,img_size,img_size,num_channels])
y_true = tf.placeholder(tf.float32, shape=[None,num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [29]:
def main_network(images, training):
    x_pretty = pt.wrap(images)
    if training:
        phase = pt.Phase.train
    else:
        phase = pt.Phase.infer
        
    with pt.defaults_scope(activation_fn=tf.nn.relu, phase=phase):
        y_pred, loss = x_pretty.\
            conv2d(kernel=filter_size1, depth=num_filters1, name='layer_conv1').\
            max_pool(kernel=2, stride=2).\
            conv2d(kernel=filter_size2, depth=num_filters2, name='layer_conv2').\
            max_pool(kernel=2, stride=2).\
            flatten().\
            fully_connected(size=dense_size, name='layer_fc1').\
            fully_connected(size=dense_size, name='layer_fc2').\
            softmax_classifier(num_classes=num_classes, labels=y_true)
        
    return  y_pred, loss

def create_network(images, training):
    with tf.variable_scope('network', reuse=not training):
         y_pred, loss = main_network(images=images, training=training)
    return y_pred, loss

In [30]:
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
_, loss = create_network(x_images, training=True)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

In [32]:
y_pred, _ = create_network(x_images, training=False)
y_pred_cls = tf.argmax(y_pred, axis=1)
correct_pred = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [34]:
session = tf.Session()

saver = tf.train.Saver()
save_dir = 'checkpoints/mnist'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
save_path = os.path.join(save_dir, 'mnist_cnn')

try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)
    saver.restore(session, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except:
    print("Failed to restore checkpoint. Initializing variables instead.")
    session.run(tf.global_variables_initializer())

Trying to restore last checkpoint ...
Failed to restore checkpoint. Initializing variables instead.


In [43]:
train_batch_size = 64

def optimize(num_iterations):
    start_time = time.time()
    
    for i in range(num_iterations):
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)
            
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        
        i_global, _ = session.run([global_step, optimizer], feed_dict=feed_dict_train)
        
        if (i_global%100 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))
            
        if (i_global%1000 == 0) or (i == num_iterations - 1):
            saver.save(session, save_path=save_path, global_step=global_step)
            print("Saved checkpoint")
            
    end_time = time.time()
    time_diff = end_time - start_time
    
    print("Time usage: " + str(timedelta(seconds=int(round(time_diff)))))


In [44]:
test_batch_size = 16

def print_test_accuracy():

    num_test = len(data.test.images)

    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = data.test.images[i:j, :]
        labels = data.test.labels[i:j, :]

        feed_dict = {x: images, y_true: labels}
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        
        i = j

    cls_true = data.test.cls
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))


In [45]:
 print_test_accuracy()

Accuracy on Test-Set: 97.0% (9696 / 10000)


In [48]:
optimize(1000)

Global Step:   1100, Training Batch Accuracy:  98.4%
Global Step:   1200, Training Batch Accuracy:  93.8%
Global Step:   1300, Training Batch Accuracy:  93.8%
Global Step:   1400, Training Batch Accuracy:  98.4%
Global Step:   1500, Training Batch Accuracy: 100.0%
Global Step:   1600, Training Batch Accuracy: 100.0%
Global Step:   1700, Training Batch Accuracy:  96.9%
Global Step:   1800, Training Batch Accuracy:  98.4%
Global Step:   1900, Training Batch Accuracy:  98.4%
Global Step:   2000, Training Batch Accuracy:  96.9%
Saved checkpoint
Time usage: 0:00:20


In [49]:
 print_test_accuracy()

Accuracy on Test-Set: 98.2% (9825 / 10000)


In [50]:
session.close()